In [ ]:
#r "nuget: Microsoft.ML, 1.4.0"
#r "nuget:  Microsoft.ML.FastTree, 1.4.0"
   
using Microsoft.ML.Data;
using System; 
using Microsoft.ML;

In [ ]:
string TRAIN_DATA_FILEPATH =  "livechat_pv_train.txt";
string MODEL_FILEPATH = "PVModel.zip";
MLContext mlContext = new MLContext(seed: 1);

In [ ]:
public class ModelInput
{
    [ColumnName("create_time"), LoadColumn(0)]
    public String Create_time { get; set; }


    [ColumnName("pv"), LoadColumn(1)]
    public float Pv { get; set; }
}

In [ ]:
public class ModelOutput
{
    public float Score { get; set; }
}

In [ ]:
public IEstimator<ITransformer> BuildTrainingPipeline(MLContext mlContext)
{
    // Data process configuration with pipeline data transformations 
    //var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("create_time_tf", "create_time")
    var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("create_time_tf", "create_time")
                              .Append(mlContext.Transforms.CopyColumns("Features", "create_time_tf"));

    // Set the training algorithm 
    var trainer = mlContext.Regression.Trainers.FastTreeTweedie(labelColumnName: "pv", featureColumnName: "Features");
    var trainingPipeline = dataProcessPipeline.Append(trainer);

    return trainingPipeline;
}

In [ ]:
public ITransformer TrainModel(MLContext mlContext, IDataView trainingDataView, IEstimator<ITransformer> trainingPipeline)
{
    Console.WriteLine("=============== Training  model ===============");

    ITransformer model = trainingPipeline.Fit(trainingDataView);

    Console.WriteLine("=============== End of training process ===============");
    return model;
}

In [ ]:
 public void CreateModel()
 {
     // Load Data
     IDataView trainingDataView = mlContext.Data.LoadFromTextFile<ModelInput>(
                                     path: TRAIN_DATA_FILEPATH,
                                     hasHeader: true,
                                     separatorChar: ',',
                                     allowQuoting: true,
                                     allowSparse: false);

     // Build training pipeline
     IEstimator<ITransformer> trainingPipeline = BuildTrainingPipeline(mlContext);

     // Evaluate quality of Model
     // Evaluate(mlContext, trainingDataView, trainingPipeline);

     // Train Model
     ITransformer mlModel = TrainModel(mlContext, trainingDataView, trainingPipeline);

     // Save model
     SaveModel(mlContext, mlModel, MODEL_FILEPATH, trainingDataView.Schema);
 }
private  void SaveModel(MLContext mlContext, ITransformer mlModel, string modelRelativePath, DataViewSchema modelInputSchema)
{
        // Save/persist the trained model to a .ZIP file
        Console.WriteLine($"=============== Saving the model  ===============");
        mlContext.Model.Save(mlModel, modelInputSchema,MODEL_FILEPATH);
        Console.WriteLine("The model is saved to {0}",MODEL_FILEPATH);
}

In [ ]:
CreateModel();

In [ ]:
string DATA_FILEPATH = "livechat_pv_test.txt";

In [ ]:
private  ModelInput CreateSingleDataSample(string dataFilePath)
{
    // Create MLContext
    MLContext mlContext = new MLContext();

    // Load dataset
    IDataView dataView = mlContext.Data.LoadFromTextFile<ModelInput>(
                                    path: dataFilePath,
                                    hasHeader: true,
                                    separatorChar: ',',
                                    allowQuoting: true,
                                    allowSparse: false);
    ModelInput sampleForPrediction = new ModelInput()
    {
        Create_time = new DateTime(2019,10,30).ToString("yyyy-MM-dd"),
        Pv = 2661
    };
    // Use first line of dataset as model input
    // You can replace this with new test data (hardcoded or from end-user application)
   // ModelInput sampleForPrediction = mlContext.Data.CreateEnumerable<ModelInput>(dataView, false)
    //                                                       .First();
    return sampleForPrediction;
}

In [ ]:
public  ModelOutput Predict(ModelInput input)
{

    // Create new MLContext
    MLContext mlContext = new MLContext();

    // Load model & create prediction engine
    string modelPath = "PVModel.zip";
    ITransformer mlModel = mlContext.Model.Load(modelPath, out var modelInputSchema);
    var predEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(mlModel);

    // Use model to make prediction on input data
    ModelOutput result = predEngine.Predict(input);
    return result;
}

In [ ]:
ModelInput sampleData = CreateSingleDataSample(DATA_FILEPATH);

// Make a single prediction on the sample data and print results
ModelOutput predictionResult = Predict(sampleData);

Console.WriteLine("Using model to make single prediction -- Comparing actual Pv with predicted Pv from sample data...\n\n");
Console.WriteLine($"create_time: {sampleData.Create_time}");
Console.WriteLine($"\n\nActual Pv: {sampleData.Pv} \nPredicted Pv: {predictionResult.Score}\n\n");
Console.WriteLine("=============== End of process, hit any key to finish ===============");


In [ ]:
Console.WriteLine("test");